In [1]:
!pip -q install kagglehub

import os, glob, json
import pandas as pd
from google.colab import userdata
import kagglehub

# read token in the Colab Secrets
os.environ["KAGGLE_API_TOKEN"] = userdata.get("KAGGLE_API_TOKEN")

HANDLE = "blacktile/starbucks-app-customer-reward-program-data"

# Download to cache content
path = kagglehub.dataset_download(HANDLE)
print("Downloaded to:", path)

# List the dataset files
files = glob.glob(path + "/**/*.json", recursive=True)
print("JSON files:", [f.split("/")[-1] for f in files])

Using Colab cache for faster access to the 'starbucks-app-customer-reward-program-data' dataset.
Downloaded to: /kaggle/input/starbucks-app-customer-reward-program-data
JSON files: ['profile.json', 'portfolio.json', 'transcript.json']


In [2]:
import glob
import pandas as pd
import json

path = "/root/.cache/kagglehub/datasets/blacktile/starbucks-app-customer-reward-program-data/versions/1"
files = glob.glob(path + "/*.json")
print(files)

def read_json_flexible(fp: str) -> pd.DataFrame:
    with open(fp, "r", encoding="utf-8") as f:
        first = f.read(1)
    if first == "[":  # JSON array: [ {...}, {...} ]
        with open(fp, "r", encoding="utf-8") as f:
            data = json.load(f)
        return pd.DataFrame(data)
    else:
        # JSON Lines: {...}
        return pd.read_json(fp, lines=True)

portfolio  = read_json_flexible(f"{path}/portfolio.json")
profile    = read_json_flexible(f"{path}/profile.json")
transcript = read_json_flexible(f"{path}/transcript.json")

print("portfolio:", portfolio.shape)
print("profile:", profile.shape)
print("transcript:", transcript.shape)

portfolio.head()

['/root/.cache/kagglehub/datasets/blacktile/starbucks-app-customer-reward-program-data/versions/1/profile.json', '/root/.cache/kagglehub/datasets/blacktile/starbucks-app-customer-reward-program-data/versions/1/transcript.json', '/root/.cache/kagglehub/datasets/blacktile/starbucks-app-customer-reward-program-data/versions/1/portfolio.json']
portfolio: (10, 6)
profile: (17000, 5)
transcript: (306534, 4)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
import pandas as pd
import numpy as np

# ==========================================
# STEP 2 & 3: Data Cleaning & Preprocessing (Final Polish)
# ==========================================

#  Helper Function: The "Duplicate Killer"
def coalesce_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    dup_names = df.columns[df.columns.duplicated()].unique()
    if len(dup_names) > 0:
        print(f"Found duplicate columns: {dup_names}, coalescing them...")
        for name in dup_names:
            cols = df.loc[:, df.columns == name]
            df[name] = cols.bfill(axis=1).iloc[:, 0]
        df = df.loc[:, ~df.columns.duplicated(keep='first')]
    return df

# ------------------------------------------
# 1. Clean Portfolio (Treatment)
# ------------------------------------------
print("Cleaning Portfolio...")
clean_portfolio = portfolio.copy()
clean_portfolio.rename(columns={'id': 'offer_id'}, inplace=True)

# One-hot encode channels
for channel in ['web', 'email', 'mobile', 'social']:
    clean_portfolio[f'channel_{channel}'] = clean_portfolio['channels'].apply(lambda x: 1 if channel in x else 0)
clean_portfolio.drop('channels', axis=1, inplace=True)

# ------------------------------------------
# 2. Clean Profile (Features)
# ------------------------------------------
print("Cleaning Profile...")
clean_profile = profile.copy()
clean_profile.rename(columns={'id': 'customer_id'}, inplace=True)
clean_profile['became_member_on'] = pd.to_datetime(clean_profile['became_member_on'], format='%Y%m%d')

# Handle Age 118 -> NaN -> Drop
# We DROP missing data to ensure valid Confounder control for Causal Inference
clean_profile['age'] = clean_profile['age'].apply(lambda x: np.nan if x == 118 else x)
clean_profile.dropna(inplace=True)

# ------------------------------------------
# 3. Clean Transcript (Outcome)
# ------------------------------------------
print("Cleaning Transcript...")

# A. Flatten & Standardize
v = pd.json_normalize(transcript['value'])
v.columns = [c.replace(" ", "_") for c in v.columns]

# B. Coalesce duplicates (Safety Net)
v = coalesce_duplicate_columns(v)

# C. Merge back
clean_transcript = pd.concat([transcript.drop(columns=['value']), v], axis=1)
clean_transcript.rename(columns={'person': 'customer_id'}, inplace=True)

# D. Filter valid users (Consistency Check)
valid_users = set(clean_profile['customer_id'])
clean_transcript = clean_transcript[clean_transcript['customer_id'].isin(valid_users)]

# (Deleted the fillna('N/A') line based on ChatGPT's advice)
# Now transactions will have real NaN in 'offer_id', which is cleaner.

print("\nData Cleaning Complete!")
print(f"Cleaned Portfolio Shape: {clean_portfolio.shape}")
print(f"Cleaned Profile Shape: {clean_profile.shape}")
print(f"Cleaned Transcript Shape: {clean_transcript.shape}")

display(clean_transcript.head())

Cleaning Portfolio...
Cleaning Profile...
Cleaning Transcript...
Found duplicate columns: Index(['offer_id'], dtype='object'), coalescing them...

Data Cleaning Complete!
Cleaned Portfolio Shape: (10, 9)
Cleaned Profile Shape: (14825, 5)
Cleaned Transcript Shape: (272762, 6)


,customer_id,event,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
5,389bc3fa690240e798340f5a15918d5c,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN


In [4]:
# Data Check
# 1) check if there are duplicate column names
dups = clean_transcript.columns[clean_transcript.columns.duplicated()].tolist()
print("Duplicated columns:", dups)

# 2) check key columns
print([c for c in clean_transcript.columns if c in ["offer_id", "amount", "reward"]])

# 3) offer_id
print(clean_transcript.groupby("event")["offer_id"].apply(lambda s: s.notna().mean()).sort_values(ascending=False))

Duplicated columns: []
['offer_id', 'amount', 'reward']
event
offer completed    1.0
offer received     1.0
offer viewed       1.0
transaction        0.0
Name: offer_id, dtype: float64


In [5]:
# ==========================================
# STEP 4: Build the User-Level Causal Dataset (Sample A, STRICT + Robust)
# ==========================================
# Strategy: "Sample A" (Strict Cohort)
# 1) Single Offer Focus: Pick the most common BOGO offer_id to avoid mixed durations.
# 2) Strict Cohort: Only users who RECEIVED this specific offer (real t0 exists).
# 3) Strict Timing: Treatment (VIEW) and Outcome (SPEND) are measured INSIDE [t0, t0 + duration].
# 4) Add key extras for causal rigor and debugging:
#    - Keep t0 in final dataset (debug, stratification, placebo later).
#    - Add t_view (first view time within window) for mechanism analysis.
#    - Add "concurrent campaigns" controls: other offers received/viewed in the same window.
#    - Add a sanity check for repeated receives of the same offer.

print("Building Strict User-Level Causal Dataset (Sample A)...")

# -----------------------------
# 0) Basic column safety checks
# -----------------------------
required_cols = {'customer_id', 'event', 'time'}
if not required_cols.issubset(set(clean_transcript.columns)):
    raise ValueError(f"clean_transcript must contain columns: {required_cols}")

# Ensure 'offer_id' exists for offer events; transactions may have NaN offer_id and that's OK.
# Ensure 'amount' exists for transaction spend; if not, create it.
if 'amount' not in clean_transcript.columns:
    clean_transcript['amount'] = 0.0

# Ensure time is numeric
clean_transcript['time'] = pd.to_numeric(clean_transcript['time'], errors='coerce')

# -----------------------------------------------
# 1) Select the single most common BOGO offer_id
# -----------------------------------------------
target_offer_type = 'bogo'

bogo_offer_ids = clean_portfolio.loc[
    clean_portfolio['offer_type'].str.lower() == target_offer_type, 'offer_id'
].unique()

df_received_all_bogo = clean_transcript[
    (clean_transcript['event'] == 'offer received') &
    (clean_transcript['offer_id'].isin(bogo_offer_ids))
].copy()

if df_received_all_bogo.empty:
    raise ValueError("No 'offer received' events found for BOGO offers. Check data cleaning and offer_id mapping.")

bogo_counts = df_received_all_bogo['offer_id'].value_counts()
target_offer_id = bogo_counts.index[0]  # Most frequent BOGO offer

# Get duration for this offer
offer_row = clean_portfolio[clean_portfolio['offer_id'] == target_offer_id]
if offer_row.empty:
    raise ValueError(f"Target offer_id {target_offer_id} not found in clean_portfolio.")
offer_duration_days = float(offer_row['duration'].iloc[0])
offer_duration_hours = int(offer_duration_days * 24)

print(f" Target Offer ID: {target_offer_id}")
print(f" Validity Duration: {offer_duration_hours} hours")

# ---------------------------------------------
# 2) Build cohort: users who RECEIVED this offer
# ---------------------------------------------
df_received_target = clean_transcript[
    (clean_transcript['event'] == 'offer received') &
    (clean_transcript['offer_id'] == target_offer_id)
].copy()

# t0 = first time they received this specific offer
cohort_users = (
    df_received_target.groupby('customer_id', as_index=False)['time']
    .min()
    .rename(columns={'time': 't0'})
)

print(f" Cohort Size (Users who received target offer at least once): {len(cohort_users)}")

# Sanity check: repeated receives of the SAME offer
recv_counts_per_user = df_received_target.groupby('customer_id')['time'].size()
pct_multi_receive = (recv_counts_per_user.gt(1).mean() * 100) if len(recv_counts_per_user) > 0 else 0.0
print(f" Users who received the same offer >1 time: {pct_multi_receive:.2f}% (MVP uses first t0)")

# ---------------------------------------------------------
# 3) Prepare cohort-level transcript (merge t0 onto events)
# ---------------------------------------------------------
# Filter transcript to only cohort users, then merge t0
cohort_user_ids = set(cohort_users['customer_id'])
df_cohort_trans = clean_transcript[clean_transcript['customer_id'].isin(cohort_user_ids)].merge(
    cohort_users, on='customer_id', how='left'
)

# Define time window boundaries per row (vectorized)
df_cohort_trans['t_end'] = df_cohort_trans['t0'] + offer_duration_hours

# ---------------------------------------------
# 4A) Treatment: viewed this offer within window
# ---------------------------------------------
view_events_in_window = df_cohort_trans[
    (df_cohort_trans['event'] == 'offer viewed') &
    (df_cohort_trans['offer_id'] == target_offer_id) &
    (df_cohort_trans['time'] >= df_cohort_trans['t0']) &
    (df_cohort_trans['time'] <= df_cohort_trans['t_end'])
].copy()

# treated_users = viewed within window
treated_users = set(view_events_in_window['customer_id'].unique())

# Also keep first view time within window (t_view) for mechanism analysis
t_view = (
    view_events_in_window.groupby('customer_id', as_index=False)['time']
    .min()
    .rename(columns={'time': 't_view'})
)

# ---------------------------------------------
# 4B) Outcome: spend within window [t0, t_end]
# ---------------------------------------------
df_transactions = df_cohort_trans[
    (df_cohort_trans['event'] == 'transaction')
][['customer_id', 'time', 'amount', 't0', 't_end']].copy()

df_transactions['amount'] = pd.to_numeric(df_transactions['amount'], errors='coerce').fillna(0.0)

spend_in_window = df_transactions[
    (df_transactions['time'] >= df_transactions['t0']) &
    (df_transactions['time'] <= df_transactions['t_end'])
].groupby('customer_id', as_index=False)['amount'].sum().rename(
    columns={'amount': 'total_spend_in_window'}
)

# ----------------------------------------------------
# 4C) Pre-treatment feature: spend in 30 days pre t0
# ----------------------------------------------------
pre_window_hours = 30 * 24  # lookback 30 days
pre_spend = df_transactions[
    (df_transactions['time'] < df_transactions['t0']) &
    (df_transactions['time'] >= df_transactions['t0'] - pre_window_hours)
].groupby('customer_id', as_index=False)['amount'].sum().rename(
    columns={'amount': 'pre_treatment_spend_30d'}
)

# --------------------------------------------------------------------
# 4D) Concurrent campaigns: other offers received/viewed in same window
# --------------------------------------------------------------------
# Rationale: window spend may be affected by other overlapping offers/campaigns.
# We add simple control features to quantify concurrent exposure/engagement.

offer_events_in_window = df_cohort_trans[
    (df_cohort_trans['event'].isin(['offer received', 'offer viewed'])) &
    (df_cohort_trans['time'] >= df_cohort_trans['t0']) &
    (df_cohort_trans['time'] <= df_cohort_trans['t_end'])
].copy()

# Other offers = offer_id != target_offer_id
offer_events_in_window['is_other_offer'] = (offer_events_in_window['offer_id'] != target_offer_id).astype(int)

# Count other-offer RECEIVED events in window
other_received = offer_events_in_window[
    (offer_events_in_window['event'] == 'offer received') &
    (offer_events_in_window['is_other_offer'] == 1)
].groupby('customer_id', as_index=False)['offer_id'].size().rename(
    columns={'size': 'other_offers_received_events_in_window'}
)

# Count other-offer VIEWED events in window
other_viewed = offer_events_in_window[
    (offer_events_in_window['event'] == 'offer viewed') &
    (offer_events_in_window['is_other_offer'] == 1)
].groupby('customer_id', as_index=False)['offer_id'].size().rename(
    columns={'size': 'other_offers_viewed_events_in_window'}
)

# Optional: unique other offers (sometimes more interpretable than raw event count)
other_received_unique = offer_events_in_window[
    (offer_events_in_window['event'] == 'offer received') &
    (offer_events_in_window['is_other_offer'] == 1)
].groupby('customer_id', as_index=False)['offer_id'].nunique().rename(
    columns={'offer_id': 'other_offers_received_unique_in_window'}
)

other_viewed_unique = offer_events_in_window[
    (offer_events_in_window['event'] == 'offer viewed') &
    (offer_events_in_window['is_other_offer'] == 1)
].groupby('customer_id', as_index=False)['offer_id'].nunique().rename(
    columns={'offer_id': 'other_offers_viewed_unique_in_window'}
)

# ---------------------------------------------
# 5) Final assembly (profile + cohort + targets)
# ---------------------------------------------
final_df = clean_profile[clean_profile['customer_id'].isin(cohort_user_ids)].copy()

# Keep t0 (important for debugging/stratification/placebo later)
final_df = final_df.merge(cohort_users, on='customer_id', how='left')

# Merge outcomes and features
final_df = final_df.merge(spend_in_window, on='customer_id', how='left')
final_df = final_df.merge(pre_spend, on='customer_id', how='left')
final_df = final_df.merge(t_view, on='customer_id', how='left')

# Merge concurrent-campaign controls
final_df = final_df.merge(other_received, on='customer_id', how='left')
final_df = final_df.merge(other_viewed, on='customer_id', how='left')
final_df = final_df.merge(other_received_unique, on='customer_id', how='left')
final_df = final_df.merge(other_viewed_unique, on='customer_id', how='left')

# Fill NaNs
final_df['total_spend_in_window'] = final_df['total_spend_in_window'].fillna(0.0)
final_df['pre_treatment_spend_30d'] = final_df['pre_treatment_spend_30d'].fillna(0.0)

final_df['other_offers_received_events_in_window'] = final_df['other_offers_received_events_in_window'].fillna(0).astype(int)
final_df['other_offers_viewed_events_in_window'] = final_df['other_offers_viewed_events_in_window'].fillna(0).astype(int)
final_df['other_offers_received_unique_in_window'] = final_df['other_offers_received_unique_in_window'].fillna(0).astype(int)
final_df['other_offers_viewed_unique_in_window'] = final_df['other_offers_viewed_unique_in_window'].fillna(0).astype(int)

# Treatment indicator (TOT/engaged): viewed inside window
final_df['treatment'] = final_df['customer_id'].isin(treated_users).astype(int)

# Instrument placeholder (in Sample A everyone received this target offer by design)
# This is included for future extensions and consistent naming across samples.
final_df['received_offer'] = 1

print(" Strict Causal Dataset Built (Sample A)!")
print(f"Final Shape: {final_df.shape}")
print(f"Treated (Viewed inside window): {final_df['treatment'].sum()}")
print(f"Control (Received but NOT viewed inside window): {len(final_df) - final_df['treatment'].sum()}")

# ---------------------------------------------
# 6) Logic checks (diagnostic only, not causal)
# ---------------------------------------------
print("\n🔍 Logic Check (Average Spend within window):")
avg_spend_treated = final_df.loc[final_df['treatment'] == 1, 'total_spend_in_window'].mean()
avg_spend_control = final_df.loc[final_df['treatment'] == 0, 'total_spend_in_window'].mean()
print(f"  - VIEWED (treated) avg spend: ${avg_spend_treated:.2f}")
print(f"  - NOT VIEWED (control) avg spend: ${avg_spend_control:.2f}")
print(f"  - Naive difference (not causal): ${avg_spend_treated - avg_spend_control:.2f}")

print("\n Quick sanity: concurrent campaigns (mean counts):")
print(final_df[['other_offers_received_events_in_window',
                'other_offers_viewed_events_in_window',
                'other_offers_received_unique_in_window',
                'other_offers_viewed_unique_in_window']].mean())

display(final_df.head())


Building Strict User-Level Causal Dataset (Sample A)...
 Target Offer ID: 9b98b8c7a33c4b65b9aebfe6a799e6d9
 Validity Duration: 168 hours
 Cohort Size (Users who received target offer at least once): 5542
 Users who received the same offer >1 time: 18.59% (MVP uses first t0)
 Strict Causal Dataset Built (Sample A)!
Final Shape: (5542, 15)
Treated (Viewed inside window): 2920
Control (Received but NOT viewed inside window): 2622

🔍 Logic Check (Average Spend within window):
  - VIEWED (treated) avg spend: $37.68
  - NOT VIEWED (control) avg spend: $17.89
  - Naive difference (not causal): $19.79

 Quick sanity: concurrent campaigns (mean counts):
other_offers_received_events_in_window    0.804042
other_offers_viewed_events_in_window      0.319379
other_offers_received_unique_in_window    0.784915
other_offers_viewed_unique_in_window      0.316853
dtype: float64


,gender,age,customer_id,became_member_on,income,t0,total_spend_in_window,pre_treatment_spend_30d,t_view,other_offers_received_events_in_window,other_offers_viewed_events_in_window,other_offers_received_unique_in_window,other_offers_viewed_unique_in_window,treatment,received_offer
0,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,408,23.22,53.79,NaN,1,0,1,0,0,1
1,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,0,37.67,0.00,6.0,1,0,1,0,1,1
2,M,68.0,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,408,18.42,39.31,420.0,1,1,1,1,1,1
3,M,65.0,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,168,0.00,9.54,192.0,1,0,1,0,1,1
4,M,57.0,6445de3b47274c759400cd68131d91b4,2017-12-31,42000.0,336,1.14,4.33,NaN,1,1,1,1,0,1


In [6]:
# ==========================================
# STEP 5: Save the Gold Standard Dataset
# ==========================================

output_csv = "final_bogo_dataset_sampleA.csv"
final_df.to_csv(output_csv, index=False)
print(f" Saved: {output_csv}")

# Optional but useful for modeling
output_parquet = "final_bogo_dataset_sampleA.parquet"
final_df.to_parquet(output_parquet, index=False)
print(f" Saved: {output_parquet}")

from google.colab import files
files.download(output_csv)
files.download(output_parquet)

print("\nQuick Peek at Spending Distribution (Window Spend):")
print(final_df.groupby('treatment')['total_spend_in_window'].describe())


 Saved: final_bogo_dataset_sampleA.csv
 Saved: final_bogo_dataset_sampleA.parquet


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Quick Peek at Spending Distribution (Window Spend):
            count       mean        std  min    25%     50%      75%     max
treatment                                                                   
0          2622.0  17.891506  41.830149  0.0  0.000   6.185  22.7625  881.35
1          2920.0  37.677226  61.331492  0.0  8.565  24.445  50.6550  999.55
